In [ ]:
import random

###Class Zone is defined here which has service time and item range for each zone
##Service time has mean and standard deviation which is flexible andn given uniform distribution
class Zone():
    def __init__(self, serviceTime, itemRange):
        self.mean, self.std = serviceTime  
        self.itemRange = itemRange
        
    def serviceTime(self):
        return int(random.uniform(self.mean - self.std, self.mean + self.std))

###Class Warehouse is defined which has zones, item types, service times, consolidation time, packaging time
##Items in each zone will be assigned equally
class Warehouse():
    # serviceTimes = (mean, std)
    def __init__(self, zones, itemTypes, serviceTimes, consolidationTime, packagingTime):
        self.itemTypes = itemTypes
        self.itemsInEachZone = itemTypes // zones
        self.consolidationTime = consolidationTime
        self.packagingTime = packagingTime
        self.initializeZones(zones, itemTypes, serviceTimes)
        
    def initializeZones(self, zones, itemTypes, serviceTimes):
        itemStart = 0
        self.zones = []
        for i in range(zones):
            z = Zone(serviceTimes, (itemStart * i, itemStart * i + self.itemsInEachZone))
            itemStart = self.itemsInEachZone
            self.zones.append(z)
            
    def fetchItem(self, item, zoneServiceTimes):
        zoneId = item // self.itemsInEachZone
        zone = self.zones[zoneId]
        if zoneId not in zoneServiceTimes:
            zoneServiceTimes[zoneId] = zone.serviceTime()
        return zoneServiceTimes[zoneId]
        
 #Order time here will be taken from poisson distribution when orders will arrive randomly   
    def fetchOrderProcessingTime(self, order, orderTime):
        zoneServiceTimes = {}
        maxFetchTime = 0
        for item in order:
            t = self.fetchItem(item, zoneServiceTimes)
            maxFetchTime = max(maxFetchTime, t)
        return maxFetchTime + self.packagingTime + self.consolidationTime + orderTime

    
#Simulation for warehouse 
## Below mentioned parameters can be changed according to the warehouse size and specifications and number of simulations is basically the cross validation process
numSimulations = 1
zones = 4
itemTypes = 40
serviceTimes = (10, 5)
consolidationTime = 5
packagingTime = 5
numOrders = 10
maxItemsInOrder = 10

simulations = []

#Orders are getting created for simulation 
def createOrders(numOrders, maxItemsInOrder, itemTypes):
    orders = []
    for _ in range(numOrders):
        order = []
        numItems = random.randint(1, maxItemsInOrder)
        for _ in range(numItems):
            item = random.randint(0, itemTypes - 1)
            order.append(item)
        orders.append(order)
    return orders

#Need to create function here for order arrival time according to the posisson distribution(Currently not done)
def calculateOrderTime():
    return 0

for simulation in range(numSimulations):
    times = []
    warehouse = Warehouse(zones, itemTypes, serviceTimes, consolidationTime, packagingTime)
    orders = createOrders(numOrders, maxItemsInOrder, itemTypes)
    print(simulation, orders)
    for order in orders:
        orderTime = calculateOrderTime()
        timeTaken = warehouse.fetchOrderProcessingTime(order, orderTime) #orderTime here is possion arrival
        times.append(timeTaken)
    
    simulations.append(times)

print(simulations)